### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Quran With Flashcards/Api Key/quran-with-flashcards-firebase-adminsdk-t8xhw-202e2c7561.json")  # Quran with Flashcards
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
df_28 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,yok çok değil o kadar da değil,4267,4268,yok çok değil o kadar da değil,IGJVw1HjIHc,https://www.youtube.com/watch?v=IGJVw1HjIHc&t=...
1,yok yok yok sen onu değil bunu,5166,5168,yok yok yok sen onu değil bunu al,t4G3MB0ULX4,https://www.youtube.com/watch?v=t4G3MB0ULX4&t=...
2,ama bu o kadar şey değil evet,394,397,ama bu o kadar şey değil evet,U4CqkDvAE_Q,https://www.youtube.com/watch?v=U4CqkDvAE_Q&t=...
3,da var yok yok daha çok var,2037,2039,yarına da var yok yok daha çok var,zXR_pFRSE7s,https://www.youtube.com/watch?v=zXR_pFRSE7s&t=...
4,var gibi de ama yok gibi de,64,68,var gibi de ama yok gibi de,ghAr1nU391c,https://www.youtube.com/watch?v=ghAr1nU391c&t=64s
...,...,...,...,...,...,...
195,ben seni daha çok,21,22,ben seni daha çok,ZsPLU4hufVQ,https://www.youtube.com/watch?v=ZsPLU4hufVQ&t=21s
196,ben seni daha çok,3356,3358,ben seni daha çok,qKdeiL8Zrsg,https://www.youtube.com/watch?v=qKdeiL8Zrsg&t=...
197,bu da bir şey ama,4751,4753,tabii bu da bir şey ama,wBjDafmCd_c,https://www.youtube.com/watch?v=wBjDafmCd_c&t=...
198,yok hayır bir şey,885,887,yok hayır bir şey yokl,5f6UWoANodU,https://www.youtube.com/watch?v=5f6UWoANodU&t=...


In [9]:
df_200 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
1,bilmiyorum sen ne demek istiyorsun ben de onu ...,721,726,bilmiyorum sen ne demek istiyorsun ben de onu ...,ZdeFof81xKs,https://www.youtube.com/watch?v=ZdeFof81xKs&t=...
2,var da bu bir sorun olur mu benim için ben bun...,7920,7926,var da bu bir sorun olur mu benim için ben bun...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
3,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...
4,gerçekten çok teşekkür ederim ya bu çok büyük ...,2257,2262,gerçekten çok teşekkür ederim ya bu çok büyük ...,a8Dov7B7alQ,https://www.youtube.com/watch?v=a8Dov7B7alQ&t=...
...,...,...,...,...,...,...
195,gerek yok çok teşekkür ederim ben zaten birkaç,186,190,gerek yok çok teşekkür ederim ben zaten birkaç...,zpfhmsZB26w,https://www.youtube.com/watch?v=zpfhmsZB26w&t=...
196,senin için ne demek olduğunu çok iyi biliyorum,7727,7730,polisliğin senin için ne demek olduğunu çok iy...,pWZnIAh9MXo,https://www.youtube.com/watch?v=pWZnIAh9MXo&t=...
197,ben gerçekten her şey için çok teşekkür ederim,875,878,ben gerçekten her şey için çok teşekkür ederim,d3LdJzLdpCI,https://www.youtube.com/watch?v=d3LdJzLdpCI&t=...
198,ki bu aslında bana iyi geldi çünkü o zaman şey,368,371,ki bu aslında bana iyi geldi çünkü o zaman şey...,X2DdZUtPDoU,https://www.youtube.com/watch?v=X2DdZUtPDoU&t=...


In [10]:
df_1000 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,https://www.youtube.com/watch?v=eqkDognLtTc&t=...
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,https://www.youtube.com/watch?v=N4YDaU51lvI&t=...
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,https://www.youtube.com/watch?v=cUAQrHbbpZ8&t=...
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...
...,...,...,...,...,...,...
195,beni daha çok seviyor ben dedim ki hayır beni ...,368,373,o dedi ki resulullah beni daha çok seviyor ben...,yqj1HmYDs3w,https://www.youtube.com/watch?v=yqj1HmYDs3w&t=...
196,bilmiyorum sen ne demek istiyorsun ben de onu ...,721,726,bilmiyorum sen ne demek istiyorsun ben de onu ...,ZdeFof81xKs,https://www.youtube.com/watch?v=ZdeFof81xKs&t=...
197,hep aynı aman boş ver sen beni asıl sen nasıls...,2444,2450,hep aynı aman boş ver sen beni asıl sen nasıls...,m6k2GOeUHNI,https://www.youtube.com/watch?v=m6k2GOeUHNI&t=...
198,siz buradan artık yok olur musunuz lütfen bir ...,5000,5005,siz buradan artık yok olur musunuz lütfen bir ...,bt7g4VoVBoE,https://www.youtube.com/watch?v=bt7g4VoVBoE&t=...


In [11]:
df_5000 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(200)
df_5000

FileNotFoundError: [Errno 2] No such file or directory: 'Turkish_Word_Group_Max_In_Youtube_Sentence_5000_Word6.csv'

In [ ]:
df_10000 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(200)
df_10000

In [ ]:
df_20000 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(200)
df_20000

In [ ]:
df_40000 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(200)
df_40000

#### Sending Data To Firestore

##### Language Data

In [14]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [15]:
fire_db.collection("language-data").document("languages").set(language_data)

update_time {
  seconds: 1669117012
  nanos: 522078000
}

##### Language Detail

##### Absolute Beginner

In [ ]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [ ]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [ ]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [ ]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [ ]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [ ]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [ ]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "This description"

In [ ]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [ ]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1 Level", "videos":A1_data_list}
#data_A1

##### A2 Level

In [ ]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [ ]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "This description"

In [ ]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [ ]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2 Level", "videos":A2_data_list}
#data_A2

##### B1 Level

In [ ]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [ ]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "This description"

In [ ]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [ ]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1 Level", "videos":B1_data_list}
#data_B1

##### B2 Level

In [ ]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [ ]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "This description"

In [ ]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [ ]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2 Level", "videos":B2_data_list}
#data_B2

##### C1 Level

In [ ]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [ ]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "This description"

In [ ]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [ ]:
data_C1 = {"description":f"{C1_description_text}", "level":4, "levelName":"C1 Level", "videos":C1_data_list}
#data_C1

##### C2 Level

In [ ]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [ ]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "This description"

In [ ]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [ ]:
data_C2 = {"description":f"{C2_description_text}", "level":4, "levelName":"C2 Level", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [ ]:
data_all = {"datas":[data_beginner,data_A1,data_A2,data_B1,data_B2,data_C1,data_C2]}
data_all

In [ ]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_28 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv")
df_28

In [ ]:
df_200 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv")
df_200

In [ ]:
df_1000 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv")
df_1000